In [1]:
#from pyspark.sql import SparkSession
import findspark
findspark.init('/usr/lib/spark-2.3.1')
from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("My App")
sc = SparkContext(conf = conf)

In [2]:
sc

<SparkContext master=local[*] appName=My App>

In [18]:
import numpy as np
#import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import Normalizer

from distkeras.transformers import LabelIndexTransformer
from distkeras.predictors import ModelPredictor
from distkeras.trainers import SingleTrainer
from distkeras.trainers import AEASGD
from distkeras.trainers import DOWNPOUR
#from distkeras.workers import NetworkWorker

from pyspark.sql.types import *
import pandas as pd
from sklearn import datasets


In [4]:
from pyspark import SQLContext
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [5]:
num_workers=4

iris = datasets.load_iris()
X = pd.DataFrame(iris.data)
X["lbl"]= iris.target
df = sqlContext.createDataFrame(X) #.createDataFrame(X)
df.printSchema()

features = list(X.columns.values)
n_features = len(features) - 1
n_classes = X['lbl'].nunique()
print('Colnames: ' + str(features))
print('Number of features ' + str(n_features))
print('Number of classes ' + str(n_classes))
features = features[0:4]
(train, test) = df.randomSplit([0.7, 0.3])

test.cache()
train.cache()
n_test = test.count()
n_train = train.count()
print('NUmber of test points: ' + str(n_test))
print('NUmber of train points: ' + str(n_train))

root
 |-- 0: double (nullable = true)
 |-- 1: double (nullable = true)
 |-- 2: double (nullable = true)
 |-- 3: double (nullable = true)
 |-- lbl: long (nullable = true)

Colnames: [0, 1, 2, 3, 'lbl']
Number of features 4
Number of classes 3
NUmber of test points: 44
NUmber of train points: 106


In [7]:
features = df.columns
features.remove('lbl')
features

['0', '1', '2', '3']

In [8]:
vector_assembler = VectorAssembler(inputCols=features, outputCol="features")
train = vector_assembler.transform(train)
test = vector_assembler.transform(test)

In [9]:
train.show()

+---+---+---+---+---+-----------------+
|  0|  1|  2|  3|lbl|         features|
+---+---+---+---+---+-----------------+
|4.3|3.0|1.1|0.1|  0|[4.3,3.0,1.1,0.1]|
|4.4|2.9|1.4|0.2|  0|[4.4,2.9,1.4,0.2]|
|4.4|3.2|1.3|0.2|  0|[4.4,3.2,1.3,0.2]|
|4.5|2.3|1.3|0.3|  0|[4.5,2.3,1.3,0.3]|
|4.6|3.1|1.5|0.2|  0|[4.6,3.1,1.5,0.2]|
|4.6|3.2|1.4|0.2|  0|[4.6,3.2,1.4,0.2]|
|4.6|3.4|1.4|0.3|  0|[4.6,3.4,1.4,0.3]|
|4.7|3.2|1.3|0.2|  0|[4.7,3.2,1.3,0.2]|
|4.8|3.0|1.4|0.3|  0|[4.8,3.0,1.4,0.3]|
|4.9|2.5|4.5|1.7|  2|[4.9,2.5,4.5,1.7]|
|4.9|3.1|1.5|0.1|  0|[4.9,3.1,1.5,0.1]|
|4.9|3.1|1.5|0.1|  0|[4.9,3.1,1.5,0.1]|
|4.9|3.1|1.5|0.1|  0|[4.9,3.1,1.5,0.1]|
|5.0|2.0|3.5|1.0|  1|[5.0,2.0,3.5,1.0]|
|5.0|2.3|3.3|1.0|  1|[5.0,2.3,3.3,1.0]|
|5.0|3.0|1.6|0.2|  0|[5.0,3.0,1.6,0.2]|
|5.0|3.3|1.4|0.2|  0|[5.0,3.3,1.4,0.2]|
|5.0|3.4|1.5|0.2|  0|[5.0,3.4,1.5,0.2]|
|5.0|3.4|1.6|0.4|  0|[5.0,3.4,1.6,0.4]|
|5.0|3.5|1.3|0.3|  0|[5.0,3.5,1.3,0.3]|
+---+---+---+---+---+-----------------+
only showing top 20 rows



In [10]:
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
train_norm = normalizer.transform(train)
test_norm = normalizer.transform(test)

train_norm.cache()
test_norm.cache()
train_norm = train_norm.repartition(num_workers)
test_norm = test_norm.repartition(num_workers)

In [11]:
from distkeras.transformers import *
encoder = OneHotTransformer(n_classes, input_col="lbl", output_col="label")
train_norm = encoder.transform(train_norm)
test_norm = encoder.transform(test_norm)
test_norm.show()

+---+---+---+---+---+-----------------+--------------------+---------------+
|  0|  1|  2|  3|lbl|         features|        normFeatures|          label|
+---+---+---+---+---+-----------------+--------------------+---------------+
|5.7|3.8|1.7|0.3|  0|[5.7,3.8,1.7,0.3]|[0.49565217391304...|[1.0, 0.0, 0.0]|
|6.1|3.0|4.9|1.8|  2|[6.1,3.0,4.9,1.8]|[0.38607594936708...|[0.0, 0.0, 1.0]|
|7.2|3.6|6.1|2.5|  2|[7.2,3.6,6.1,2.5]|[0.37113402061855...|[0.0, 0.0, 1.0]|
|6.3|2.7|4.9|1.8|  2|[6.3,2.7,4.9,1.8]|[0.40127388535031...|[0.0, 0.0, 1.0]|
|5.7|3.0|4.2|1.2|  1|[5.7,3.0,4.2,1.2]|[0.40425531914893...|[0.0, 1.0, 0.0]|
|5.4|3.7|1.5|0.2|  0|[5.4,3.7,1.5,0.2]|[0.5,0.3425925925...|[1.0, 0.0, 0.0]|
|4.8|3.4|1.9|0.2|  0|[4.8,3.4,1.9,0.2]|[0.46601941747572...|[1.0, 0.0, 0.0]|
|4.8|3.0|1.4|0.1|  0|[4.8,3.0,1.4,0.1]|[0.51612903225806...|[1.0, 0.0, 0.0]|
|5.4|3.9|1.3|0.4|  0|[5.4,3.9,1.3,0.4]|[0.49090909090909...|[1.0, 0.0, 0.0]|
|5.6|2.5|3.9|1.1|  1|[5.6,2.5,3.9,1.1]|[0.42748091603053...|[0.0, 1.0, 0.0]|

In [12]:
model = Sequential()
model.add(Dense(150, input_shape=(n_features,)))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.6))
model.add(Dense(80))
model.add(Activation('relu'))
model.add(Dense(n_classes))
model.add(Activation('softmax'))
model.summary()
#model.get_config()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 150)               750       
_________________________________________________________________
activation_1 (Activation)    (None, 150)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               15100     
_________________________________________________________________
activation_2 (Activation)    (None, 100)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 80)                8080      
__________

In [13]:
optimizer = 'adagrad'
loss = 'categorical_crossentropy'
results = {}
time_spent = {}

In [14]:
train_norm = train_norm.select([c for c in train_norm.columns if c in ['normFeatures','label']])
train_norm.show()

+--------------------+---------------+
|        normFeatures|          label|
+--------------------+---------------+
|[0.39743589743589...|[0.0, 0.0, 1.0]|
|[0.38953488372093...|[0.0, 0.0, 1.0]|
|[0.41353383458646...|[0.0, 1.0, 0.0]|
|[0.38607594936708...|[0.0, 0.0, 1.0]|
|[0.43103448275862...|[0.0, 1.0, 0.0]|
|[0.37113402061855...|[0.0, 0.0, 1.0]|
|[0.39428571428571...|[0.0, 0.0, 1.0]|
|[0.42857142857142...|[0.0, 1.0, 0.0]|
|[0.42953020134228...|[0.0, 1.0, 0.0]|
|[0.46728971962616...|[1.0, 0.0, 0.0]|
|[0.39655172413793...|[0.0, 0.0, 1.0]|
|[0.43037974683544...|[0.0, 1.0, 0.0]|
|[0.49019607843137...|[1.0, 0.0, 0.0]|
|[0.41791044776119...|[0.0, 1.0, 0.0]|
|[0.42241379310344...|[0.0, 1.0, 0.0]|
|[0.43650793650793...|[0.0, 1.0, 0.0]|
|[0.5,0.3333333333...|[1.0, 0.0, 0.0]|
|[0.51612903225806...|[1.0, 0.0, 0.0]|
|[0.41216216216216...|[0.0, 1.0, 0.0]|
|[0.40127388535031...|[0.0, 0.0, 1.0]|
+--------------------+---------------+
only showing top 20 rows



In [19]:
trainer = SingleTrainer(keras_model=model, loss=loss, worker_optimizer=optimizer, 
                        features_col="normFeatures",num_epoch=10,
                        batch_size=10, metrics=["accuracy"])
trained_model = trainer.train(train_norm)
trainer.get_training_time()

In [24]:
predictor = ModelPredictor(keras_model=trained_model, features_col="normFeatures")
test_set = predictor.predict(test_norm)
test_set.show()

+---+---+---+---+---+-----------------+--------------------+---------------+--------------------+
|  0|  1|  2|  3|lbl|         features|        normFeatures|          label|          prediction|
+---+---+---+---+---+-----------------+--------------------+---------------+--------------------+
|5.7|3.8|1.7|0.3|  0|[5.7,3.8,1.7,0.3]|[0.49565217391304...|[1.0, 0.0, 0.0]|[0.98926639556884...|
|6.1|3.0|4.9|1.8|  2|[6.1,3.0,4.9,1.8]|[0.38607594936708...|[0.0, 0.0, 1.0]|[0.00568212661892...|
|7.2|3.6|6.1|2.5|  2|[7.2,3.6,6.1,2.5]|[0.37113402061855...|[0.0, 0.0, 1.0]|[0.00362483831122...|
|6.3|2.7|4.9|1.8|  2|[6.3,2.7,4.9,1.8]|[0.40127388535031...|[0.0, 0.0, 1.0]|[0.00469665555283...|
|5.7|3.0|4.2|1.2|  1|[5.7,3.0,4.2,1.2]|[0.40425531914893...|[0.0, 1.0, 0.0]|[0.02224191278219...|
|5.4|3.7|1.5|0.2|  0|[5.4,3.7,1.5,0.2]|[0.5,0.3425925925...|[1.0, 0.0, 0.0]|[0.99150168895721...|
|4.8|3.4|1.9|0.2|  0|[4.8,3.4,1.9,0.2]|[0.46601941747572...|[1.0, 0.0, 0.0]|[0.98028093576431...|
|4.8|3.0|1.4|0.1|  0

In [25]:
locDF = pd.DataFrame(test_set.collect())
locDF

,0,1,2,3,4,5,6,7,8
0,5.7,3.8,1.7,0.3,0,"[5.7, 3.8, 1.7, 0.3]","[0.4956521739130435, 0.33043478260869563, 0.14...","[1.0, 0.0, 0.0]","[0.9892663955688477, 0.00991078931838274, 0.00..."
1,6.1,3.0,4.9,1.8,2,"[6.1, 3.0, 4.9, 1.8]","[0.38607594936708856, 0.18987341772151897, 0.3...","[0.0, 0.0, 1.0]","[0.005682126618921757, 0.44963324069976807, 0...."
2,7.2,3.6,6.1,2.5,2,"[7.2, 3.6, 6.1, 2.5]","[0.3711340206185567, 0.18556701030927836, 0.31...","[0.0, 0.0, 1.0]","[0.003624838311225176, 0.4021952450275421, 0.5..."
3,6.3,2.7,4.9,1.8,2,"[6.3, 2.7, 4.9, 1.8]","[0.40127388535031844, 0.17197452229299362, 0.3...","[0.0, 0.0, 1.0]","[0.004696655552834272, 0.43822696805000305, 0...."
4,5.7,3.0,4.2,1.2,1,"[5.7, 3.0, 4.2, 1.2]","[0.40425531914893625, 0.21276595744680854, 0.2...","[0.0, 1.0, 0.0]","[0.02224191278219223, 0.5851777791976929, 0.39..."
5,5.4,3.7,1.5,0.2,0,"[5.4, 3.7, 1.5, 0.2]","[0.5, 0.34259259259259256, 0.13888888888888887...","[1.0, 0.0, 0.0]","[0.9915016889572144, 0.007895910181105137, 0.0..."
6,4.8,3.4,1.9,0.2,0,"[4.8, 3.4, 1.9, 0.2]","[0.46601941747572817, 0.3300970873786408, 0.18...","[1.0, 0.0, 0.0]","[0.9802809357643127, 0.01790972612798214, 0.00..."
7,4.8,3.0,1.4,0.1,0,"[4.8, 3.0, 1.4, 0.1]","[0.5161290322580645, 0.32258064516129037, 0.15...","[1.0, 0.0, 0.0]","[0.9906836748123169, 0.008643537759780884, 0.0..."
8,5.4,3.9,1.3,0.4,0,"[5.4, 3.9, 1.3, 0.4]","[0.49090909090909085, 0.3545454545454545, 0.11...","[1.0, 0.0, 0.0]","[0.9921557307243347, 0.007300983648747206, 0.0..."
9,5.6,2.5,3.9,1.1,1,"[5.6, 2.5, 3.9, 1.1]","[0.42748091603053434, 0.19083969465648856, 0.2...","[0.0, 1.0, 0.0]","[0.019714094698429108, 0.5847409963607788, 0.3..."


In [27]:
npar = np.double(locDF[8][0])

score = 0
for i in range(locDF[4].size-1):
    #print(i)
     
    if locDF[4][i] == np.argmax(np.double(locDF[8][i])):
        score = score + 1
    #print(str(locDF[4][i]) + '  ' + str(np.double(locDF[8][i])) + '  score: ' + str(score))

print('Accuracy score: ' + str(100*(score/locDF[4].size))) 

Accuracy score: 95.45454545454545
